#### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("..")

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

#### Test for CUDA

In [3]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('No GPU, training on CPU')
    dev = torch.device('cpu')
else:
    print('GPU found, training on GPU')
    dev = torch.device('cuda')

No GPU, training on CPU


#### Load MNIST

In [4]:
## Make sure batch_size = 1 for now!!

def load_mnist(batch_size=1, shuffle_train=True):
    transform = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5,), (0.5,))])
    train_set = torchvision.datasets.MNIST("../data", train=True, download=True, transform=transform)
    test_set = torchvision.datasets.MNIST("../data", train=False, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=shuffle_train)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [5]:
mnist_train_loader, mnist_test_loader = load_mnist()

In [6]:
image_batch, label_batch = next(iter(mnist_train_loader))

In [7]:
n_classes = 10

In [8]:
input_dim = 784
hidden_dim = 100
output_dim = 1

In [9]:
## Gating vector
tvec = torch.zeros(n_classes,hidden_dim)
for ii in range(n_classes):
    t_half = torch.randint(0, 2, (1, hidden_dim//2)).float()*2 - 1
    tvec[ii,::2] = t_half
    tvec[ii,1::2] = -t_half

In [10]:
## change to appropriate shapes!!
image_batch = torch.squeeze(image_batch).view(1,-1)
image_batch = image_batch.repeat(n_classes,1)
label_batch = F.one_hot(label_batch,n_classes).view(-1,1)

#### Architecture

In [11]:
class RNNModule:
    """An RNN cell responsible for a single timestep.

    Args:
        inp_dim (int): Input size.
        hid_dim (int): Hidden size.
        out_dim (int): Output size.
    """
    def __init__(self, inp_dim, hid_dim, out_dim,t_vec):
        self.inp_dim = inp_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.tvec = t_vec
        n_classes = t_vec.shape[0] ## NOTE: Edit as needed!!

        ## Wih, Whh, Woh are the parameters, so we set requires_grad=True
        self.Wih = torch.empty(hid_dim, inp_dim, requires_grad=True)
        self.Whh = torch.empty(hid_dim, hid_dim, requires_grad=True)
        self.Woh = torch.empty(out_dim, hid_dim, requires_grad=True)

        ## These are the gradients on Wih, Whh, and Woh computed manually
        ## Will be compared to the gradients computed by PyTorch
        self.Wih_grad = torch.zeros_like(self.Wih)
        self.Whh_grad = torch.zeros_like(self.Whh)
        self.Woh_grad = torch.zeros_like(self.Woh)
        
        self.Wih_grad_geb = torch.zeros_like(self.Wih)
        self.Whh_grad_geb = torch.zeros_like(self.Whh)
        self.Woh_grad_geb = torch.zeros_like(self.Woh)
        
        self.Wih_grad_all = {}
        self.Whh_grad_all = {}
        self.Woh_grad_all = {}
        
        self.Wih_grad_geb_all = {}
        self.Whh_grad_geb_all = {}
        self.Woh_grad_geb_all = {}
        
        self.reset_parameters()
    
    def reset_parameters(self):
        """Initialize parameters.

        The parameters will be initialized from the uniform
        distribution U(-0.1, 0.1).
        """
        s = 0.05  # larger value may make the gradients explode/vanish
        torch.nn.init.uniform_(self.Wih, -s, s)
        torch.nn.init.uniform_(self.Whh, 0, s)
        torch.nn.init.uniform_(self.Woh, 0, s)
        
    def zero_grad(self):
        """Set the gradients to zero."""
        self.Wih_grad.zero_()
        self.Whh_grad.zero_()
        self.Woh_grad.zero_()
        
        self.Wih_grad_geb.zero_()
        self.Whh_grad_geb.zero_()
        self.Woh_grad_geb.zero_()

    def forward(self, x, hp, kk):
        """Perform the forward computation.
        
        Args:
            x (Tensor): Input at the current timestep.
            hp (Tensor): Hidden state at the previous timestep.
            
        Returns:
            Tensor: Output at the current timestep.
            Tensor: Hidden state at the current timestep.
        """
        _, _, h, _, y = self._get_internals(x, hp, kk)
        return y, h

    def backward(self, y_grad, rn_grad, x, hp, kk):
        """Perform the backward computation.
        
        Args:
            y_grad (Tensor): Gradient on output at the current timestep.
            rn_grad (Tensor): Gradient on vector r at the next timestep.
            x (Tensor): Input at the current timestep that was passed to `forward`.
            hp (Tensor): Hidden state at the previous timestep that was passed to `forward`.
            
        Returns:
            Tensor: Gradient on vector r at the current timestep.
        """
        n_classes = y_grad.shape[0]
        
        # Get internal vectors r, h, and s from forward computation
        rint, r, h, s, _ = self._get_internals(x, hp, kk)

        ## BPTT calculations
        s_grad = y_grad * torch.sigmoid(s) * (1-torch.sigmoid(s)) ## note manual differentiation!!
        h_grad = self.Woh.t().matmul(s_grad) + self.Whh.t().matmul(rn_grad)
        r_grad = h_grad * ((self.tvec[kk]*r)>0)*1 ## note manual differentiation!!
        rint_grad = r_grad*((self.tvec[kk]*rint)>0)*1 ## note manual differentiation
        
        # Parameter gradients are accumulated
        self.Wih_grad += rint_grad.view(-1, 1).matmul(x.view(1, -1))
        self.Whh_grad += r_grad.view(-1, 1).matmul(hp.view(1, -1))
        self.Woh_grad += s_grad.view(-1, 1).matmul(h.view(1, -1))

        self.Wih_grad_all[kk] = r_grad.view(-1, 1).matmul(x.view(1, -1))
        self.Whh_grad_all[kk] = r_grad.view(-1, 1).matmul(hp.view(1, -1))
        self.Woh_grad_all[kk] = s_grad.view(-1, 1).matmul(h.view(1, -1))
        
        ## R-GEB calculation
        
        ## Indicator of post synaptic activity
        ## Pre-synaptic activation
        ## Global error vector

        x_y_grad = y_grad*x
        rint_post = ((self.tvec[kk]*rint)>0)*1.
        
        hp_y_grad = y_grad*hp
        r_post = ((self.tvec[kk]*r)>0)*1.
        
        self.Wih_grad_geb += torch.outer(rint_post,x_y_grad) #done!
        self.Whh_grad_geb += torch.outer(r_post,hp_y_grad) #done! 
        self.Woh_grad_geb += s_grad.view(-1,1).matmul(h.view(1, -1)) #done!
        
        self.Wih_grad_geb_all[kk] = torch.outer(rint_post,x_y_grad)
        self.Whh_grad_geb_all[kk] = torch.outer(r_post,hp_y_grad)
        self.Woh_grad_geb_all[kk] = s_grad.view(-1,1).matmul(h.view(1, -1))

        return r_grad
    
    def _get_internals(self, x, hp, kk):
        # Actual forward computations
        rint = self.Wih.matmul(x)
        r = ((self.tvec[kk]*rint)>0)*rint + self.Whh.matmul(hp)
        h = ((self.tvec[kk]*r)>0)*r
        s = self.Woh.matmul(h)
        y = torch.sigmoid(s)
        
        return rint, r, h, s, y

In [12]:
class RNN:
    def __init__(self, cell):
        self.cell = cell
    
    def forward(self, xs, h0):
        """Perform the forward computation for all timesteps.
        
        Args:
            xs (Tensor): 2-D tensor of inputs for each timestep. The
                first dimension corresponds to the number of timesteps.
            h0 (Tensor): Initial hidden state.
            
        Returns:
            Tensor: 2-D tensor of outputs for each timestep. The first
                dimension corresponds to the number of timesteps.
            Tensor: 2-D tensor of hidden states for each timestep plus
                `h0`. The first dimension corresponds to the number of
                timesteps.
        """
        ys, hs = [], [h0]
        for ii, x in enumerate(xs):
            y, h = self.cell.forward(x, hs[-1],ii)
            ys.append(y)
            hs.append(h)
        return torch.stack(ys), torch.stack(hs)
    
    def backward(self, ys_grad, xs, hs):
        """Perform the backward computation for all timesteps.
        
        Args:
            ys_grad (Tensor): 2-D tensor of the gradients on outputs
                for each timestep. The first dimension corresponds to
                the number of timesteps.
            xs (Tensor): 2-D tensor of inputs for each timestep that
                was passed to `forward`.
            hs (Tensor): 2-D tensor of hidden states that is returned
                by `forward`.
        """
        # For the last timestep, the gradient on r is zero
        rn_grad = torch.zeros(self.cell.hid_dim)

        for ii, (y_grad, x, hp) in enumerate(reversed(list(zip(ys_grad, xs, hs)))):
            rn_grad  = cell.backward(y_grad, rn_grad, x, hp, n_classes-ii-1)

In [13]:
cell = RNNModule(input_dim, hidden_dim, output_dim, tvec)
rnn = RNN(cell)

#### Loss

In [14]:
# def compute_cross_entropy(predictions, targets, epsilon=1e-15):
#     """
#     Computes cross entropy between targets (encoded as one-hot vectors)
#     and predictions. 
#     Input: predictions (N, k) ndarray
#            targets (N, k) ndarray        
#     Returns: scalar
#     """
#     predictions = torch.clip(predictions, epsilon, 1. - epsilon)
#     N = predictions.shape[0]
#     ce = -torch.sum(targets*torch.log(predictions+1e-12))/N
    
#     return ce

In [15]:
def compute_loss(ys, ts):
    return 0.5 * torch.sum((ys - ts)**2)

In [16]:
## Note -- changed!!
xs = image_batch
hp = torch.zeros(cell.hid_dim)
ts = label_batch

In [17]:
ys, hs = rnn.forward(xs, hp)

In [18]:
ys, hs = rnn.forward(xs, hp)
loss = compute_loss(ys, ts)

#### PyTorch gradients

In [19]:
## gradient of cross entropy w.r.t. yhat_k = (1/yhat_k)*y_k
## gradient of mse w.r.t. y_hat = y_hat - y

In [20]:
loss.backward()

In [21]:
# rnn.cell.Wih.grad

In [22]:
# rnn.cell.Whh.grad

In [23]:
# rnn.cell.Woh.grad

#### Manual gradients

In [24]:
# This is obtained from our loss function
# ys_grad = torch.matmul(label_batch,ys)
ys_grad = ys - ts

with torch.no_grad():  # required so PyTorch won't raise error
    rnn.cell.zero_grad()
    rnn.backward(ys_grad, xs, hs)

In [25]:
v1 = torch.arange(1., 101.)
v2 = torch.arange(1., 785.)
torch.outer(v1,v2).shape

torch.Size([100, 784])

In [26]:
v1.shape

torch.Size([100])

In [27]:
# rnn.cell.Wih_grad

In [28]:
# rnn.cell.Whh_grad

In [29]:
# rnn.cell.Wih_grad

In [30]:
# rnn.cell.Woh_grad - rnn.cell.Woh.grad

In [31]:
# rnn.cell.Whh_grad - rnn.cell.Whh.grad

In [32]:
# rnn.cell.Wih_grad - rnn.cell.Wih.grad

In [33]:
# torch.max(torch.abs(rnn.cell.Woh_grad - rnn.cell.Woh.grad))

In [34]:
# torch.max(torch.abs(rnn.cell.Whh_grad - rnn.cell.Whh.grad))

In [35]:
# torch.max(torch.abs(rnn.cell.Wih_grad - rnn.cell.Wih.grad))

In [36]:
# print(torch.sign(rnn.cell.Woh_grad) - torch.sign(rnn.cell.Woh_grad_geb))

In [37]:
# (torch.abs(rnn.cell.Woh_grad - rnn.cell.Woh_grad_geb))

In [38]:
print('Summed total differences for Whh: ',torch.abs(rnn.cell.Whh_grad - rnn.cell.Whh_grad_geb))

Summed total differences for Whh:  tensor([[ 5.7070,  9.4697,  3.8032,  ...,  5.8523,  3.8743, 11.8866],
        [ 7.4038,  1.4343,  0.9471,  ...,  1.2072,  7.4682,  2.4491],
        [ 0.0540,  9.5127,  0.3405,  ...,  1.9263,  0.2000, 12.8698],
        ...,
        [ 0.0540,  9.1172,  0.5374,  ...,  1.2078,  1.2888, 10.5332],
        [ 5.7074,  9.9335,  4.3793,  ...,  5.8555,  3.8745, 13.3747],
        [ 7.4038,  0.9750,  0.3730,  ...,  1.2070,  7.4677,  0.9696]])


In [39]:
print('Summed total differences for Wih: ',torch.abs(rnn.cell.Wih_grad - rnn.cell.Wih_grad_geb))

Summed total differences for Wih:  tensor([[2.9373, 2.9373, 2.9373,  ..., 2.9373, 2.9373, 2.9373],
        [2.9390, 2.9390, 2.9390,  ..., 2.9390, 2.9390, 2.9390],
        [3.6751, 3.6751, 3.6751,  ..., 3.6751, 3.6751, 3.6751],
        ...,
        [2.7510, 2.7510, 2.7510,  ..., 2.7510, 2.7510, 2.7510],
        [4.2569, 4.2569, 4.2569,  ..., 4.2569, 4.2569, 4.2569],
        [3.1666, 3.1666, 3.1666,  ..., 3.1666, 3.1666, 3.1666]])


In [40]:
for tt in range(n_classes):
    print(tt)
    
    rnn.cell.Wih_grad_all[tt][rnn.cell.Wih_grad_all[tt]==torch.clamp(rnn.cell.Wih_grad_all[tt], -1e-6 , 1e-6)]=0
    rnn.cell.Wih_grad_all[tt][rnn.cell.Wih_grad_geb_all[tt]==torch.clamp(rnn.cell.Wih_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
    rnn.cell.Whh_grad_all[tt][rnn.cell.Whh_grad_all[tt]==torch.clamp(rnn.cell.Whh_grad_all[tt], -1e-6 , 1e-6)]=0
    rnn.cell.Whh_grad_all[tt][rnn.cell.Whh_grad_geb_all[tt]==torch.clamp(rnn.cell.Whh_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
    rnn.cell.Woh_grad_all[tt][rnn.cell.Woh_grad_all[tt]==torch.clamp(rnn.cell.Woh_grad_all[tt], -1e-6 , 1e-6)]=0
    rnn.cell.Woh_grad_all[tt][rnn.cell.Woh_grad_geb_all[tt]==torch.clamp(rnn.cell.Woh_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
    print('Timestep signed differences for Wih:',torch.unique(torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt])))
    print('Timestep signed differences for Whh:',torch.unique(torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt])))
    print('Timestep signed differences for Woh:',torch.unique(torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt])))
    print('-----------------------')

0
Timestep signed differences for Wih: tensor([0.])
Timestep signed differences for Whh: tensor([0.])
Timestep signed differences for Woh: tensor([0.])
-----------------------
1
Timestep signed differences for Wih: tensor([-1.,  0.,  1.])
Timestep signed differences for Whh: tensor([0.])
Timestep signed differences for Woh: tensor([0.])
-----------------------
2
Timestep signed differences for Wih: tensor([-1.,  0.,  1.])
Timestep signed differences for Whh: tensor([0.])
Timestep signed differences for Woh: tensor([0.])
-----------------------
3
Timestep signed differences for Wih: tensor([-1.,  0.,  1.])
Timestep signed differences for Whh: tensor([0.])
Timestep signed differences for Woh: tensor([0.])
-----------------------
4
Timestep signed differences for Wih: tensor([-1.,  0.,  1.])
Timestep signed differences for Whh: tensor([0.])
Timestep signed differences for Woh: tensor([0.])
-----------------------
5
Timestep signed differences for Wih: tensor([-1.,  0.,  1.])
Timestep sign

In [41]:
for tt in range(n_classes):
    
    print(tt)
    
    totWih = np.prod(rnn.cell.Wih_grad_all[tt].shape)
    totWhh = np.prod(rnn.cell.Whh_grad_all[tt].shape)
    totWoh = np.prod(rnn.cell.Woh_grad_all[tt].shape)
    
#     rnn.cell.Wih_grad_all[tt][rnn.cell.Wih_grad_all[tt]==torch.clamp(rnn.cell.Wih_grad_all[tt], -1e-6 , 1e-6)]=0
#     rnn.cell.Wih_grad_all[tt][rnn.cell.Wih_grad_geb_all[tt]==torch.clamp(rnn.cell.Wih_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
#     rnn.cell.Whh_grad_all[tt][rnn.cell.Whh_grad_all[tt]==torch.clamp(rnn.cell.Whh_grad_all[tt], -1e-6 , 1e-6)]=0
#     rnn.cell.Whh_grad_all[tt][rnn.cell.Whh_grad_geb_all[tt]==torch.clamp(rnn.cell.Whh_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
#     rnn.cell.Woh_grad_all[tt][rnn.cell.Woh_grad_all[tt]==torch.clamp(rnn.cell.Woh_grad_all[tt], -1e-6 , 1e-6)]=0
#     rnn.cell.Woh_grad_all[tt][rnn.cell.Woh_grad_geb_all[tt]==torch.clamp(rnn.cell.Woh_grad_geb_all[tt], -1e-6 , 1e-6)]=0
    
    num0_Wih = len(np.where((torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt]))==0)[0])
    num1_Wih = len(np.where((torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt]))==1)[0])
    num2_Wih = len(np.where((torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt]))==2)[0])
    numm1_Wih = len(np.where((torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt]))==-1)[0])
    numm2_Wih = len(np.where((torch.sign(rnn.cell.Wih_grad_all[tt]) - torch.sign(rnn.cell.Wih_grad_geb_all[tt]))==-2)[0])
    
    num0_Whh = len(np.where((torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt]))==0)[0])
    num1_Whh = len(np.where((torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt]))==1)[0])
    num2_Whh = len(np.where((torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt]))==2)[0])
    numm1_Whh = len(np.where((torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt]))==-1)[0])
    numm2_Whh = len(np.where((torch.sign(rnn.cell.Whh_grad_all[tt]) - torch.sign(rnn.cell.Whh_grad_geb_all[tt]))==-2)[0])
    
    num0_Woh = len(np.where((torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt]))==0)[0])
    num1_Woh = len(np.where((torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt]))==1)[0])
    num2_Woh = len(np.where((torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt]))==2)[0])
    numm1_Woh = len(np.where((torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt]))==-1)[0])
    numm2_Woh = len(np.where((torch.sign(rnn.cell.Woh_grad_all[tt]) - torch.sign(rnn.cell.Woh_grad_geb_all[tt]))==-2)[0])
    
    print('Frac. 0 signed differences for Wih:',num0_Wih/totWih)
    print('Frac. 1 signed differences for Wih:',num1_Wih/totWih)
    print('Frac. 2 signed differences for Wih:',num2_Wih/totWih)
    print('Frac. -1 signed differences for Wih:',numm1_Wih/totWih)
    print('Frac. -2 signed differences for Wih:',numm2_Wih/totWih)
    print('-----------------------')
    print('Frac. 0 signed differences for Whh:',num0_Whh/totWhh)
    print('Frac. 1 signed differences for Whh:',num1_Whh/totWhh)
    print('Frac. 2 signed differences for Whh:',num2_Whh/totWhh)
    print('Frac. -1 signed differences for Whh:',numm1_Whh/totWhh)
    print('Frac. -2 signed differences for Whh:',numm2_Whh/totWhh)
    print('-----------------------')
    print('Frac. 0 signed differences for Woh:',num0_Woh/totWoh)
    print('Frac. 1 signed differences for Woh:',num1_Woh/totWoh)
    print('Frac. 2 signed differences for Woh:',num2_Woh/totWoh)
    print('Frac. -1 signed differences for Woh:',numm1_Woh/totWoh)
    print('Frac. -2 signed differences for Woh:',numm2_Woh/totWoh)
    print('=======================')

0
Frac. 0 signed differences for Wih: 1.0
Frac. 1 signed differences for Wih: 0.0
Frac. 2 signed differences for Wih: 0.0
Frac. -1 signed differences for Wih: 0.0
Frac. -2 signed differences for Wih: 0.0
-----------------------
Frac. 0 signed differences for Whh: 1.0
Frac. 1 signed differences for Whh: 0.0
Frac. 2 signed differences for Whh: 0.0
Frac. -1 signed differences for Whh: 0.0
Frac. -2 signed differences for Whh: 0.0
-----------------------
Frac. 0 signed differences for Woh: 1.0
Frac. 1 signed differences for Woh: 0.0
Frac. 2 signed differences for Woh: 0.0
Frac. -1 signed differences for Woh: 0.0
Frac. -2 signed differences for Woh: 0.0
1
Frac. 0 signed differences for Wih: 0.96
Frac. 1 signed differences for Wih: 0.034846938775510206
Frac. 2 signed differences for Wih: 0.0
Frac. -1 signed differences for Wih: 0.005153061224489796
Frac. -2 signed differences for Wih: 0.0
-----------------------
Frac. 0 signed differences for Whh: 1.0
Frac. 1 signed differences for Whh: 0.0
F